In [1]:
import numpy as np
import pandas as pd

import zipfile
import io
import os
import glob
import time
import re

import folium

# Bergstraße Groundwater data

Hessen provides a fairly reasonable interface for retrieving groundwater data:
https://lgd.hessen.de/mapapps/resources/apps/lgd/index.html?lang=en

The area was manually selected and then, all groundwater station data and measurement series were downloaded.

## 1. Basic cleaning of station data

In [2]:
station_path = './data/groundwater/bergstraße/stationdata_raw/gw_stations_bergstraße_raw.xlsx'
stations = pd.read_excel(station_path)#, sep=';')
stations

,Mst.-ID,Name,Kurzname,Betreiber,Ostwert,Nordwert,Rechtswert,Hochwert,TK-Blatt,Messstellenart,...,Sohltiefe,Filter-Oberkante,Filter-Unterkante,Rohrinnendurchmesser (mm),Grundwasserkörper,Hydrogeol. Teilraum,Stilllegungsdatum,Stilllegungsgrund,GewAnlagen-ID,Bohrarchiv HLNUG
0,12890,BIBLIS (alt),544047.0,Regierungspräsidium Darmstadt,458821.000,5505747.000,3458880.0,5507510.0,6216 - Gernsheim,BR,...,6.25,4.25,6.25,100,2395_3101,03101 - Rheingrabenscholle,"Aug 18, 2010",Ersatz durch Neubau,NaN,225
1,12891,GROSS-ROHRHEIM,544017.0,Regierungspräsidium Darmstadt,462960.000,5505877.000,3463020.0,5507640.0,6216 - Gernsheim,BR,...,3.75,2.75,3.75,35,2395_3101,03101 - Rheingrabenscholle,"Aug 1, 1973",Sonstiges,NaN,11
2,12892,NORDHEIM,544013.0,Regierungspräsidium Darmstadt,455982.000,5505917.000,3456040.0,5507680.0,6216 - Gernsheim,BR,...,7.00,6.00,7.00,35,2395_3101,03101 - Rheingrabenscholle,NaN,NaN,NaN,7
3,12893,GROSS-ROHRHEIM,544016.0,Regierungspräsidium Darmstadt,461080.000,5505917.000,3461140.0,5507680.0,6216 - Gernsheim,BR,...,4.90,3.70,4.80,35,2395_3101,03101 - Rheingrabenscholle,"Jun 30, 1982",Sonstiges,NaN,10
4,12894,NORDHEIM (alt),544014.0,Regierungspräsidium Darmstadt,457072.000,5505927.000,3457130.0,5507690.0,6216 - Gernsheim,BR,...,5.20,4.20,5.20,35,2395_3101,03101 - Rheingrabenscholle,"May 1, 2007",Ersatz durch Neubau,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,17129,VIERNHEIM,544273.0,NaN,467596.000,5490910.000,3467658.0,5492667.0,6417 - Mannheim Nordost,BR,...,15.00,3.00,15.00,125,2394_3101,03101 - Rheingrabenscholle,NaN,NaN,NaN,0
248,17130,LAMPERTHEIM,544274.0,Regierungspräsidium Darmstadt,465688.000,5492192.000,3465750.0,5493950.0,6417 - Mannheim Nordost,BR,...,12.80,8.00,12.00,125,2393_3101,03101 - Rheingrabenscholle,NaN,NaN,NaN,0
249,17131,VIERNHEIM,544275.0,NaN,469267.000,5491023.000,3469330.0,5492780.0,6417 - Mannheim Nordost,BR,...,11.00,7.00,11.00,125,2394_3101,03101 - Rheingrabenscholle,NaN,NaN,NaN,0
250,17740,LAMPERTHEIM,NaN,Regierungspräsidium Darmstadt,463546.000,5493941.000,3463607.0,5495700.0,6416 - Mannheim Nordwest,BR,...,15.00,9.00,15.00,125,2393_3101,03101 - Rheingrabenscholle,NaN,NaN,NaN,0


The data needs some basic cleaning before further use.

In [3]:
# drop unnecessary columns
cols_to_keep = ['Mst.-ID', 'Ostwert', 'Nordwert', 
                'Baudatum', 'Stilllegungsdatum', 
                'Geländehöhe', 'Messpunkthöhe']
stations = stations[cols_to_keep]

# assign new column names
new_col_names = {'Mst.-ID': 'station_id', 'Ostwert': 'x', 'Nordwert': 'y',
                 'Baudatum': 'start_date', 'Stilllegungsdatum': 'end_date',
                 'Geländehöhe': 'ground_elev', 'Messpunkthöhe': 'measure_elev'}
stations = stations.rename(columns=new_col_names)
stations = stations.drop_duplicates()

# change date columns to datetime type
stations['start_date'] = pd.to_datetime(stations['start_date'], format='%d.%m.%Y')
stations['end_date'] = pd.to_datetime(stations['end_date'], format='%b %d, %Y')

#fill empty end_dates with a recent date
stations['end_date'] = stations['end_date'].fillna(pd.to_datetime('2024-03-01'))

# calculate station lifespan
stations.start_date = pd.to_datetime(stations.start_date)
stations.end_date = pd.to_datetime(stations.end_date)
stations['lifespan'] = stations.end_date - stations.start_date
stations.info()

<class 'pandas.core.frame.DataFrame'>
Index: 243 entries, 0 to 251
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype          
---  ------        --------------  -----          
 0   station_id    243 non-null    int64          
 1   x             243 non-null    float64        
 2   y             243 non-null    float64        
 3   start_date    243 non-null    datetime64[ns] 
 4   end_date      243 non-null    datetime64[ns] 
 5   ground_elev   243 non-null    float64        
 6   measure_elev  243 non-null    float64        
 7   lifespan      243 non-null    timedelta64[ns]
dtypes: datetime64[ns](2), float64(4), int64(1), timedelta64[ns](1)
memory usage: 17.1 KB


In [4]:
stations.head()

,station_id,x,y,start_date,end_date,ground_elev,measure_elev,lifespan
0,12890,458821.0,5505747.0,1969-01-01,2010-08-18,87.44,87.95,15204 days
1,12891,462960.0,5505877.0,1951-01-01,1973-08-01,89.96,90.01,8248 days
2,12892,455982.0,5505917.0,1951-01-01,2024-03-01,89.10,89.40,26723 days
3,12893,461080.0,5505917.0,1951-01-01,1982-06-30,88.17,88.03,11503 days
4,12894,457072.0,5505927.0,1951-01-01,2007-05-01,88.13,88.49,20574 days


In [5]:
stations.describe()

,station_id,x,y,start_date,end_date,ground_elev,measure_elev,lifespan
count,243.000000,243.000000,2.430000e+02,243,243,243.000000,243.000000,243
mean,13906.761317,466223.891128,5.502013e+06,1970-08-09 02:45:55.555555556,2017-03-01 17:34:48.888888832,91.537160,92.719588,17006 days 14:48:53.333333248
min,12890.000000,453983.000000,5.487354e+06,1912-01-01 00:00:00,1966-01-01 00:00:00,0.000000,0.000000,0 days 00:00:00
25%,13435.000000,462939.765500,5.497358e+06,1955-01-01 00:00:00,2024-03-01 00:00:00,90.410000,90.755000,12187 days 00:00:00
50%,13596.000000,467438.000000,5.501588e+06,1974-01-01 00:00:00,2024-03-01 00:00:00,93.260000,93.470000,16861 days 00:00:00
75%,13790.500000,470237.000000,5.505902e+06,1978-01-01 00:00:00,2024-03-01 00:00:00,95.400000,95.900000,22157 days 12:00:00
max,17993.000000,474005.000000,5.514651e+06,2013-01-01 00:00:00,2024-03-01 00:00:00,127.280000,127.130000,40968 days 00:00:00
std,1235.659078,4896.366854,6.187809e+03,NaN,NaN,14.037493,11.354104,7196 days 14:16:15.050202368


Filter for lifespan > 4 years.

In [6]:
stations = stations.loc[stations.lifespan > pd.to_timedelta(4*365)]

Some stations have erroneous ground_elev values. We correct them by using this approximation: ground_elev = measure_elev - mean(measure_elev - ground_elev)

In [7]:
good_stations = stations.loc[stations.ground_elev > 80]
meas_dist_from_ground = np.mean(good_stations.measure_elev 
                                - good_stations.ground_elev).round(1)
meas_dist_from_ground


0.0

In [8]:
stations.loc[stations.ground_elev < 80, 'ground_elev'] = stations.loc[stations.ground_elev < 80, 'measure_elev'] - meas_dist_from_ground

Filter for stations with ground_elev < 100 to exclude Odenwald conditions.

In [9]:
stations = stations.loc[stations.ground_elev < 100]

## 2. Read groundwater measurement data to df

In [10]:
# Directory containing the folders
folder_path = './data/groundwater/bergstraße/stationdata_raw/'

# File name pattern to search for
file_pattern = 'Wasserstaende*.csv' 

df_list = []

# Check if the item in the directory is a folder
if os.path.isdir(folder_path):
    # Use glob to search for files matching the pattern inside the folder
    files = glob.glob(os.path.join(folder_path, file_pattern))
    
    # Process the found files
    for file in files:
        #print("Found file '{}' in folder '{}'".format(os.path.basename(file), folder))
        df = pd.read_csv(file, sep=';')
        df_list.append(df)


The data needs some basic cleaning before further use.

In [11]:
# clean column names
df = pd.concat(df_list)

# drop unnecessary columns
cols_to_keep = ['Mst.-ID', 'Datum', 'Abstich (m unter MP)', 
                'Wasserspiegel (m ü. NN)', 'Wasserspiegel (m unter GOK)',
               'Wassertemperatur (°C)']
df = df[cols_to_keep]

# assign new column names
new_col_names = {'Mst.-ID': 'station_id', 'Datum': 'date', 
                 'Abstich (m unter MP)': 'water_dist', 
                 'Wasserspiegel (m ü. NN)': 'water_level', 
                 'Wasserspiegel (m unter GOK)': 'water_depth',
                'Wassertemperatur (°C)': 'water_temp'}
df = df.rename(columns=new_col_names)

# change date column to datetime type
df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y')

Several rows have erroneous entries (water_level==0, water_temp==0); replace them and related measurements with nan.

In [12]:
# if water_level == 0, entire measurement is corrupted
df.loc[df['water_level'] == 0, ['water_dist', 'water_level', 'water_depth', 'water_temp']] = np.nan

# it water_temp == 0, no temp measurement available
df.loc[df['water_temp'] == 0, ['water_temp']] = np.nan

In [13]:
df.describe()

,station_id,date,water_dist,water_level,water_depth,water_temp
count,578813.000000,578813,575651.000000,575651.000000,575651.000000,66890.00000
mean,13576.277737,1995-01-30 17:50:26.388833536,4.008921,89.598707,3.632829,12.27388
min,12890.000000,1912-10-15 00:00:00,0.000000,81.140000,-0.870000,2.02000
25%,13134.000000,1980-08-11 00:00:00,2.520000,86.940000,2.140000,11.30000
50%,13556.000000,1997-09-22 00:00:00,3.420000,89.420000,3.100000,12.00000
75%,13773.000000,2013-07-15 00:00:00,4.520000,92.250000,4.180000,12.90000
max,17794.000000,2024-03-20 00:00:00,28.160000,102.710000,28.310000,21.30000
std,603.023430,NaN,2.979896,3.129021,2.927834,1.55057


## Check integrity

Verify that both dfs contain the same unique station_ids.

In [14]:
stations = stations.loc[stations.station_id.isin(df.station_id.unique())]
df = df.loc[df.station_id.isin(stations.station_id.unique())]

In [15]:
stations.describe()

,station_id,x,y,start_date,end_date,ground_elev,measure_elev,lifespan
count,214.000000,214.000000,2.140000e+02,214,214,214.000000,214.000000,214
mean,13695.831776,466120.712121,5.501813e+06,1968-07-19 06:10:05.607476632,2017-04-27 23:19:37.570093568,92.942523,93.362383,17814 days 17:09:31.962616832
min,12890.000000,453983.000000,5.487354e+06,1912-01-01 00:00:00,1966-01-01 00:00:00,86.450000,86.500000,134 days 00:00:00
25%,13437.500000,462975.000000,5.497298e+06,1954-04-02 06:00:00,2024-03-01 00:00:00,90.542500,90.860000,15240 days 00:00:00
50%,13594.500000,467208.000000,5.501563e+06,1973-12-28 12:00:00,2024-03-01 00:00:00,93.170000,93.435000,17592 days 00:00:00
75%,13782.000000,469934.500000,5.505762e+06,1978-01-01 00:00:00,2024-03-01 00:00:00,95.295000,95.760000,23711 days 12:00:00
max,17794.000000,473615.000000,5.514651e+06,2013-01-01 00:00:00,2024-03-01 00:00:00,99.920000,100.560000,40968 days 00:00:00
std,821.378796,4790.694434,6.067589e+03,NaN,NaN,2.945119,3.001687,6962 days 15:07:51.718233088


In [16]:
df.describe()

,station_id,date,water_dist,water_level,water_depth,water_temp
count,557260.000000,557260,554273.000000,554273.000000,554273.000000,66890.00000
mean,13589.102144,1995-03-01 05:35:27.078921728,3.594004,89.469690,3.220631,12.27388
min,12890.000000,1912-10-15 00:00:00,0.000000,81.140000,-0.870000,2.02000
25%,13157.000000,1980-08-25 00:00:00,2.500000,86.900000,2.100000,11.30000
50%,13568.000000,1997-12-01 00:00:00,3.360000,89.240000,3.030000,12.00000
75%,13774.000000,2013-08-22 00:00:00,4.400000,92.090000,4.040000,12.90000
max,17794.000000,2024-03-20 00:00:00,11.160000,96.270000,10.790000,21.30000
std,609.112889,NaN,1.527469,3.005891,1.476485,1.55057


## Write data to files

In [17]:
stations.to_csv('./data/groundwater/bergstraße/gw_stations_bergstraße.csv', index=False)

In [18]:
df.to_csv('./data/groundwater/bergstraße/gw.csv', index=False)